In [1]:
import numpy as np
import re
match_number = re.compile('-?\ *[0-9]+\.?[0-9]*(?:[Ee]\ *-?\ *[0-9]+)?')
s_filter  = ' '.join(re.findall(r"\((\d+)\)",s)))

In [2]:
'''
this is a program for calculating the time derivatives of the veloctity field. 

as this isn't supported by openfoam post-processsing, we'll start with a simple method of just delta_u/delta_t

we'll read in the lists of values for the uniform mesh at each timestep and then subtract the previous timestep's ones

'''

"\nthis is a program for calculating the time derivatives of the veloctity field. \n\nas this isn't supported by openfoam post-processsing, we'll start with a simple method of just delta_u/delta_t\n\nwe'll read in the lists of values for the uniform mesh at each timestep and then subtract the previous timestep's ones\n\n"

In [79]:
def getU(t):
    '''
    for a given timestep, this method extracts the velocity data from the openfoam file
    returns the velocity data in a numpy array, where each entry is the velocity in a cell
    '''
    t_string = "{:.1f}".format(t)
    
    #removes trailing 0 and decimal point to be consistent with openfoam
    #directory naming structure
    if t_string[-1] == '0':
        t_string = t_string[:-2]
        
    filename = t_string+'/U'
    internal_mesh = []
    outlet_boundary = []
    
    with open(filename) as u_file:
        
        while True:
            line = u_file.readline()
            
            if line.startswith('internalField'):
                break
                
        
        next(u_file)
        next(u_file)
        
        #this section gets the values of the internal mesh cells
        while(True):
            line = u_file.readline()
            if "boundaryField" in line:
                break
            else:
                #this is a rather hacky way of dealing with the closing bracket of the internal mesh
                #list of vectors
                if len(line) > 2:
                    #this finds all numbers enclosed in brackets
                    internal_mesh.append(re.search(r'\((.*?)\)',line).group(1))
                    
        
        #this section gets the values of the rear boundary
        while(True):
            
            line = u_file.readline()
            if "rightWall" in line:
                #skips to the relevant values
                next(u_file)
                next(u_file)
                next(u_file)
                boundary_line = u_file.readline()
                #use a regex to get a list of all the cell vector values at the boundary
                boundary_values = re.findall('\(.*?\)',boundary_line) #all of the values at the boundary
                boundary_cellVals = [re.findall(match_number, u) for u in boundary_values if re.findall(match_number, u)] #vectors of each of the cells
                outlet_boundary = [[float(u) for u in v] for v in boundary_cellVals]
                break
            
    #print((u_list))
        
    for i in range(len(internal_mesh)-1, -1, -1):
        
        internal_mesh[i] = internal_mesh[i].strip().split(' ')
        internal_mesh[i] = [re.findall(match_number, u)[0] for u in internal_mesh[i] if re.findall(match_number, u)]
        internal_mesh[i] = [float(u) for u in internal_mesh[i] if u]
        
        if internal_mesh[i] == []:
            del internal_mesh[i]

    
    internal_mesh_array = np.array([np.array(u) for u in internal_mesh])
    outlet_boundary_array = np.array([np.array(u) for u in outlet_boundary])
    
    return internal_mesh_array, outlet_boundary_array

In [80]:
def calc_dUdt(t, delta_t):
    '''this method returns a crude approximation of the time derivatives delta u/ delta t
    
    returns an array with the differences
    '''
    
    t2 = t
    t1 = max(0, t2-delta_t)
    u2_internal, u2_boundary = getU(t2)
    u1_internal, u1_boundary = getU(t1)
    
    dUdt_internal = (u2_internal - u1_internal)/delta_t
    dUdt_boundary = (u2_boundary - u1_boundary)/delta_t
    print(len(dUdt_internal))
    
    print(dUdt_internal)

In [81]:
calc_dUdt(2, 0.1)

294
[[ 1.887000e-04  3.310000e-05  0.000000e+00]
 [-1.873600e-06  1.377000e-06  0.000000e+00]
 [ 3.770000e-04 -7.600000e-06  0.000000e+00]
 [ 3.463000e-04  3.602000e-04  3.002000e-31]
 [-3.733400e-06  9.093400e-07 -4.239400e-34]
 [-1.895600e-06 -2.314700e-06  9.777000e-34]
 [ 1.094500e-05  3.244000e-06 -3.375600e-32]
 [-4.637000e-07 -5.828000e-06 -3.030000e-36]
 [ 1.837000e-07 -2.468000e-07  8.515400e-36]
 [ 4.812000e-07 -2.071000e-07  7.938700e-36]
 [-1.016100e-06 -8.422000e-07 -3.168000e-33]
 [ 4.040000e-08 -1.633300e-07 -5.123000e-33]
 [ 5.291000e-07  3.656800e-07 -7.725000e-33]
 [ 1.332000e-07  3.966800e-07  0.000000e+00]
 [ 1.233900e-06 -5.547000e-07  0.000000e+00]
 [ 2.715000e-07  1.429300e-06  1.391900e-33]
 [ 6.050000e-04 -5.460000e-04  0.000000e+00]
 [ 7.490000e-04 -6.450000e-04  3.267000e-31]
 [-7.000000e-05 -6.290000e-04  0.000000e+00]
 [-1.650000e-04 -7.240000e-04  0.000000e+00]
 [-3.300000e-04 -3.739000e-04 -1.802000e-31]
 [-3.800000e-04 -3.991000e-04  1.664000e-31]
 [-3.7